In [6]:
pip install chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [8]:
import chromedriver_autoinstaller
from selenium import webdriver
import time

class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
#         options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.set_window_size(1600, 1100)

        return driver

In [9]:
driver = Driver.driver_init()

In [10]:
def find_product_paths(urls):
    product_paths = []
    
    ## 상의/자켓/하의 분류 루프
    for (url, pages_cnt) in urls:
        ## 각 옷 분류의 페이지 갯수 루프
        for page_num in range(pages_cnt):
            try:
                driver.get(url + "&page=" + str(page_num+1))
            except:
                continue

            products = driver.find_elements('xpath', '//ul[@class="prdNeo prd_set"]/li')
            products_cnt = len(products)

            ## 각 product의 path를 product_paths 리스트에 저장
            for i in range(products_cnt):
                product = driver.find_elements('xpath', '//ul[@class="prdNeo prd_set"]/li')[i]
                product_path = product.find_element('xpath', './div/div[@class="thumbnail on"]/a').get_attribute('href')
                product_paths.append(product_path)
                
    return product_paths

In [19]:
def find_all_color_paths(product_paths):
    all_color_paths = []
    
    for product_path in product_paths:
        ## 기존의 product_path 추가
        all_color_paths.append(product_path)
        
        try:
            driver.get(product_path)
        except:
            continue
            
        other_color_products = driver.find_elements('xpath','//div[@id="product_relation"]/div[@class="xans-element- xans-product xans-product-relationlist relate_thumbs_img noSlide"]/ul/li')
        other_products_cnt = len(other_color_products)
        
        ## 다른 색들의 product_path 추가
        for i in range(other_products_cnt):
            other_product = driver.find_elements('xpath','//div[@id="product_relation"]/div[@class="xans-element- xans-product xans-product-relationlist relate_thumbs_img noSlide"]/ul/li')[i]
            other_product_path = other_product.find_element('xpath', './div/a').get_attribute('href')
            all_color_paths.append(other_product_path)
    
    return all_color_paths
            

In [ ]:
def crawl_pages(product_paths):
    
    base_dict= {"baseCat":[], "baseName": [], "baseCode": [], "baseUrl": [], "colorName": [],
            "baseGender": [], "price": [], "basePrice": [], "colorFabric": [], "imageUrl": [],
            "size": [], "isAvailable": []
            }
    
    ## 드라이버 초기화
    driver = Driver.driver_init()
    ## 각 product에 대한 정보 추출
    for product_path in product_paths:
        driver.get(product_path)

        cat = 'top'
        if "cate_no=30" in product_path or "cate_no=61" in product_path:
            cat = 'top'
        elif "cate_no=31" in product_path:
            cat = 'bottom'

        baseURL = product_path
        base_gender = 0
        text_info = driver.find_element('xpath', '//div[@class="detail_top_right"]')
#         imageURL = driver.find_element('xpath','//img[@class="BigImage "]').get_attribute('src')

        base_name = text_info.find_element('xpath', './div[@class="prd_name_wrap"]/h2').text
        color_name = base_name.split(" ")[-1]
        base_code = base_name.replace(color_name, "")[:-1]

        base_price = ""
        price = ""
        color_fabric = ""
        trs = text_info.find_elements("xpath",'./div[2]/table/tbody/tr') #body의 table 항목 갯수                       
        if len(trs) == 5:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = text_info.find_element("xpath",'.//span[@id="span_product_price_sale"]').text
            color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[5]/td/span').text
        elif len(trs) == 4:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = base_price
            color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[4]/td/span').text
        elif len(trs) == 3:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = base_price
            color_fabric = ""
        elif len(trs) == 2:
            base_price = ""
            price = ""
            color_fabric = ""
            

        size_elements = text_info.find_elements("xpath",'.//optgroup[@label="사이즈"]/option')
        size = "M"
        is_available = True

        for element in size_elements:
            size = element.text.replace(" [품절]","")
            if "[품절]" in element.text:
                is_available = False
            else:
                is_available = True

            base_dict["baseCat"].append(cat)
            base_dict["baseName"].append(base_name)
            base_dict["baseCode"].append(base_code)
            base_dict["baseUrl"].append(baseURL)
            base_dict["colorName"].append(color_name)
            base_dict["baseGender"].append(0)
            base_dict["price"].append(price)
            base_dict["basePrice"].append(base_price)
            base_dict["colorFabric"].append(color_fabric)
            base_dict["imageUrl"].append(imageURL)
            base_dict["size"].append(size)
            base_dict["isAvailable"].append(is_available)
            
    return base_dict

In [12]:
import pandas as pd

In [13]:
urls = [
        ('https://whoisnerdy.com/product/list.html?cate_no=30', 15), #상의
        ('https://whoisnerdy.com/product/list.html?cate_no=61', 5), #자켓
        ('https://whoisnerdy.com/product/list.html?cate_no=31', 7), #하의
         ]

product_paths = find_product_paths(urls)
print(len(product_paths))

777


In [20]:
all_color_paths = find_all_color_paths(product_paths)
print(len(all_color_paths))

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.112)
Stacktrace:
	GetHandleVerifier [0x00007FF6199A52A2+57122]
	(No symbol) [0x00007FF61991EA92]
	(No symbol) [0x00007FF6197EE3AB]
	(No symbol) [0x00007FF6197DBA47]
	(No symbol) [0x00007FF6197DB6C0]
	(No symbol) [0x00007FF6197EFA71]
	(No symbol) [0x00007FF61985E27F]
	(No symbol) [0x00007FF619846DB3]
	(No symbol) [0x00007FF61981D2B1]
	(No symbol) [0x00007FF61981E494]
	GetHandleVerifier [0x00007FF619C4EF82+2849794]
	GetHandleVerifier [0x00007FF619CA1D24+3189156]
	GetHandleVerifier [0x00007FF619C9ACAF+3160367]
	GetHandleVerifier [0x00007FF619A36D06+653702]
	(No symbol) [0x00007FF61992A208]
	(No symbol) [0x00007FF6199262C4]
	(No symbol) [0x00007FF6199263F6]
	(No symbol) [0x00007FF6199167A3]
	BaseThreadInitThunk [0x00007FFAFA1C26AD+29]
	RtlUserThreadStart [0x00007FFAFB20AA68+40]
